In [1]:
from __future__ import print_function
import argparse
from fastai.vision import *
from glob import glob
from matplotlib import pyplot as plt
import cv2
import random
import numpy as np
from numpy.linalg import inv
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_path = '/home/jupyter/.fastai/data/train2017'
validation_path = '/home/jupyter/.fastai/data/val2017'
test_path = '/home/jupyter/.fastai/data/test2017'

# The following code needs optimization
def ImagePreProcessing(image,path):
    
    img = cv2.imread(path+'/%s'%image,0)
    img = cv2.resize(img,(320,240))

    rho          = 32
    patch_size   = 128
    top_point    = (32,32)
    left_point   = (patch_size+32, 32)
    bottom_point = (patch_size+32, patch_size+32)
    right_point  = (32, patch_size+32)
    test_image = img.copy()
    four_points = [top_point, left_point, bottom_point, right_point]

    perturbed_four_points = []
    for point in four_points:
        perturbed_four_points.append((point[0] + random.randint(-rho,rho), point[1]+random.randint(-rho,rho)))

    H = cv2.getPerspectiveTransform( np.float32(four_points), np.float32(perturbed_four_points) )
    H_inverse = inv(H)

    warped_image = cv2.warpPerspective(img,H_inverse, (320,240))

    annotated_warp_image = warped_image.copy()

    Ip1 = test_image[top_point[1]:bottom_point[1],top_point[0]:bottom_point[0]]
    Ip2 = warped_image[top_point[1]:bottom_point[1],top_point[0]:bottom_point[0]]

    training_image = np.dstack((Ip1, Ip2))
    H_four_points = np.subtract(np.array(perturbed_four_points), np.array(four_points))
    datum = (training_image, H_four_points)
    
    return datum

In [13]:
def savedata(path):
    lst = os.listdir(path+'/')
    os.makedirs(path+'processed/')
    new_path = path+'processed/'
    for i in lst:
        np.save(new_path+'%s'%i[0:12],ImagePreProcessing(i,path))
        
savedata(train_path)
savedata(validation_path)
savedata(test_path)

FileExistsError: [Errno 17] File exists: '/home/jupyter/.fastai/data/train2017processed/'

In [43]:
# Creating our own customized dataset class in pytorch
 
class CocoDdataset(Dataset):
    def __init__(self,path):
        X=()
        Y=()
        lst = os.listdir(path)
        it=0
        for i in lst:
            array = np.load(path+'%s'%i)
            x = torch.from_numpy((array[0].astype(float)-127.5)/127.5)
            X = X+(x,)
            y = torch.from_numpy(array[1].astype(float) / 32.)
            Y = Y+(y,)
            it+=1
        self.len = it
        self.X_data = X
        self.Y_data = Y
    def __getitem__(self,index):
        return self.X_data[index], self.Y_data[index] 
    def __len__(self):
        return self.len
    
    

            


In [ ]:
train_path = '/home/jupyter/.fastai/data/train2017processed/'
validation_path = '/home/jupyter/.fastai/data/val2017processed/'
test_path = '/home/jupyter/.fastai/data/test2017processed/'

TrainingData = CocoDdataset(train_path)
ValidationData = CocoDdataset(validation_path)
TestData = CocoDdataset(test_path)

TrainLoader = DataLoader(TrainingData,batch_size=64)
ValidationLoader = DataLoader(ValidationData,batch_size=64)
TestLoader = DataLoader(TestData,batch_size=64)



In [ ]:

class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(2,64,3,padding=1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU())                           
        self.layer2 = nn.Sequential(nn.Conv2d(64,64,3,padding=1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(nn.Conv2d(64,64,3,padding=1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU())
        self.layer4 = nn.Sequential(nn.Conv2d(64,64,3,padding=1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2))
        self.layer5 = nn.Sequential(nn.Conv2d(64,128,3,padding=1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU())        
        self.layer6 = nn.Sequential(nn.Conv2d(128,128,3,padding=1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2))
        self.layer7 = nn.Sequential(nn.Conv2d(128,128,3,padding=1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU())
        self.layer8 = nn.Sequential(nn.Conv2d(128,128,3,padding=1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU())
        self.drop1 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128*16*16,1024)
        self.drop2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024,8)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.drop1(out)
        out = out.reshape(out.size(0),-1)
        out = self.fc1(out)
        out = self.drop2(out)
        out = self.fc2(out)
        return out
        
 
 



In [ ]:
model = Model().to(device)

summary(model,(2, 128, 128))




In [ ]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(),lr=0.005, momentum=0.9)
#as in paper 
num_samples = 118287
total_iteration = 90000
batch_size = 64
steps_per_epoch = num_samples / batch_size
epochs = int(total_iteration / steps_per_epoch)
model = Model().to(device)
optimizer = optim.SGD(model.parameters(),lr=0.005, momentum=0.9)
checkpoint = torch.load('model1.pth')
model.load_state_dict(checkpoint['state_dict'])
optimizer = optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']
for epoch in range(2):
    
    for i, (images, target) in enumerate(TrainLoader):
        images = images.to(device)
        target = target.to(device)
        images = images.view(-1,2,128,128)
        images = images.float()
        target = target.float()
        outputs = model(images)
        loss = criterion(outputs, target.view(-1,8))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 512 == 0:
            print('Train Epoch: [{}/{}] [{}/{} ({:.0f}%)]\Mean Squared Error: {:.6f}'.format(
                epoch+1,epochs, i * len(images), len(TrainLoader),
                100. * i / len(TrainLoader), loss))

In [90]:
state = {'epoch': 48, 'state_dict': model.state_dict(),
             'optimizer': optimizer.state_dict() }
torch.save(state, 'model1.pth')

In [91]:
file_name = "model2.pth"
torch.save(model.state_dict(), file_name)

In [107]:
model.eval()
for epoch in range(epochs):
    with torch.no_grad():
        
        for i,(images, target) in enumerate(ValidationLoader):

            # compute output
            images = (images.view(-1,2,128,128)).to(device)
            target = target.to(device)
            images = images.float()
            target = target.float()
            print(target.shape)
            print(images.shape)
            outputs = model(images)
            print(outputs.shape)
            #_, prediction = torch.max(outputs.data)
            print(prediction.shape)

            # measure accuracy and record loss
            loss = criterion(outputs, target.view(64,8))
            if (i+1) % 64 == 0:
                print('Train Epoch: [{}/{}] [{}/{} ({:.0f}%)]\Mean Squared Error: {:.6f}'.format(
                epoch+1,epochs, i * len(images), len(ValidationLoader),
                100. * i / len(ValidationLoader), loss))

torch.Size([64, 4, 2])
torch.Size([64, 2, 128, 128])
torch.Size([64, 8])
torch.Size([64])
torch.Size([64, 4, 2])
torch.Size([64, 2, 128, 128])
torch.Size([64, 8])
torch.Size([64])
torch.Size([64, 4, 2])
torch.Size([64, 2, 128, 128])
torch.Size([64, 8])
torch.Size([64])
torch.Size([64, 4, 2])
torch.Size([64, 2, 128, 128])
torch.Size([64, 8])
torch.Size([64])
torch.Size([64, 4, 2])
torch.Size([64, 2, 128, 128])
torch.Size([64, 8])
torch.Size([64])
torch.Size([64, 4, 2])
torch.Size([64, 2, 128, 128])
torch.Size([64, 8])
torch.Size([64])
torch.Size([64, 4, 2])
torch.Size([64, 2, 128, 128])
torch.Size([64, 8])
torch.Size([64])
torch.Size([64, 4, 2])
torch.Size([64, 2, 128, 128])
torch.Size([64, 8])
torch.Size([64])
torch.Size([64, 4, 2])
torch.Size([64, 2, 128, 128])
torch.Size([64, 8])
torch.Size([64])
torch.Size([64, 4, 2])
torch.Size([64, 2, 128, 128])
torch.Size([64, 8])
torch.Size([64])
torch.Size([64, 4, 2])
torch.Size([64, 2, 128, 128])
torch.Size([64, 8])
torch.Size([64])
torch.Size

RuntimeError: shape '[64, 8]' is invalid for input of size 64

In [24]:
model = Model().to(device)
optimizer = optim.SGD(model.parameters(),lr=0.005, momentum=0.9)
checkpoint = torch.load('model1.pth')
model.load_state_dict(checkpoint['state_dict'])
optimizer = optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']



In [22]:
model.eval()

AttributeError: 'NoneType' object has no attribute 'eval'

In [39]:
from PIL import Image
array = np.load('/home/jupyter/.fastai/data/val2017processed/000000435206.npy')
x = torch.from_numpy((array[0].astype(float)))
y = torch.from_numpy(array[1].astype(float) /32 )
x=x.to(device)
y=y.to(device)
x = x.float()
y = y.float()

output=model(x.view(-1,2,128,128))

In [40]:
print((output*32).view(4,2))

tensor([[ 39.3952,  -1.3436],
        [  6.4070,  18.0036],
        [-10.4787, -15.2772],
        [ 40.0358,   2.5350]], device='cuda:0', grad_fn=<ViewBackward>)


In [41]:
print((y*32))

tensor([[12., 23.],
        [-9., 23.],
        [29., 21.],
        [10., -1.]], device='cuda:0')


In [7]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import mat_, prediction = torch.max(outputs.data, 1)plotlib.pyplot as plt

import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from PIL import Image
from torch.autograd import Variable
torch.load('model.pth')
model.eval()
array = np.load('/home/jupyter/.fastai/data/val2017processed/000000435205.npy')
x = torch.from_numpy((array[0].astype(float)-127.5)/127.5)
y = torch.from_numpy(array[1].astype(float) )
